#### 資料來源: https://github.com/darrenjhsu/birdid  
1.   483252 , 0.8 , 0.1 , 0.1
2.   此檔案為裁切圖片寫入雲端硬碟，下一份檔案才為模型運算

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
datapath = '/content/gdrive/My Drive/Bird_ID_project/nabirds'

In [ ]:
if __name__ == '__main__':
    file_dir = '/content/gdrive/My Drive/Bird_ID_project/nabirds/'
    images = pd.read_csv(file_dir + 'images.txt', sep=" ", header=None, names = ["image_id","image_name"])
    print(images)
    print("read image")
    train_test_split = pd.read_csv(file_dir + 'train_test_split.txt', sep=" ", header=None, names = ["image_id","is_training"])
    print("read split")
    image_size = pd.read_csv(file_dir + 'sizes.txt', sep=" ", header=None, names = ["image_id","image_width","image_height"])
    print("read image size")
    classes = pd.read_csv(file_dir + 'classes.txt', sep=" ", header=None, names = ["class_id", "class_name"])
    print("read classes")
    image_class_label = pd.read_csv(file_dir + 'image_class_labels.txt', sep=" ", header=None, names = ["image_id","class_id"])
    print("read image class label")
    hierarchy = pd.read_csv(file_dir + 'hierarchy.txt', sep=" ", header=None, names = ["child_class_id","parent_class_id"])
    print("read hierarchy")    
    bounding_box = pd.read_csv(file_dir + 'bounding_boxes.txt', sep=" ", header=None, names = ["image_id","bb_x","bb_y","bb_width","bb_height"])
    print("read bounding boxes")
    
    img_bbox = pd.merge(images, bounding_box, on="image_id")
    img_class_bbox = pd.merge(img_bbox,image_class_label,on="image_id")
    full_frame = pd.merge(img_class_bbox,classes,on="class_id")
    
    print(full_frame)
    

                                   image_id                                 image_name
0      0000139e-21dc-4d0c-bfe1-4cae3c85c829  0817/0000139e21dc4d0cbfe14cae3c85c829.jpg
1      0000d9fc-4e02-4c06-a0af-a55cfb16b12b  0860/0000d9fc4e024c06a0afa55cfb16b12b.jpg
2      00019306-9d83-4334-b255-a447742edce3  0900/000193069d834334b255a447742edce3.jpg
3      0001afd4-99a1-4a67-b940-d419413e23b3  0645/0001afd499a14a67b940d419413e23b3.jpg
4      000332b8-997c-4540-9647-2f0a8495aecf  0929/000332b8997c454096472f0a8495aecf.jpg
...                                     ...                                        ...
48557  fff86e8b-795f-400a-91e8-565bbb8c453a  0891/fff86e8b795f400a91e8565bbb8c453a.jpg
48558  fff926d7-ccad-4788-839e-97af2dd99372  0660/fff926d7ccad4788839e97af2dd99372.jpg
48559  fffa33ef-a765-408d-8d66-6efc7f504c71  0492/fffa33efa765408d8d666efc7f504c71.jpg
48560  ffff0d87-bc84-4ef2-a47e-a4bfa48502ce  0372/ffff0d87bc844ef2a47ea4bfa48502ce.jpg
48561  fffff3a5-2a75-47d0-887f-03871e3f9a37

In [ ]:
if __name__ == '__main__':
    WBNH = full_frame[full_frame['class_name'] == 'White-breasted_Nuthatch']
    NCAM = full_frame[full_frame['class_name'] == 'Northern_Cardinal_(Adult_Male)']
    
    for index, row in WBNH.iterrows():
        if (index == 29669):
            print(row['image_name'])
            img = file_dir + 'images/' + row['image_name']
            fig,ax = plt.subplots(1)
            image = plt.imread(img)
            plt.imshow(image)
            rect = patches.Rectangle((row['bb_x'],row['bb_y']),row['bb_width'],row['bb_height'],linewidth=1,edgecolor='r',facecolor='none')
            ax.add_patch(rect)
            plt.show()

In [ ]:
def paint_to_square(img, z):     #將圖片其他部分變成黑色
    desired_size = 224           #模型需要的大小
    old_size = img.shape[:2] # (height, width) 

    ratio = float(desired_size)/max(old_size)           #計算縮放比率
    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format

    im = cv2.resize(img, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]              #計算填黑的邊框
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)    #整數除法(去除小數點)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)   #將周圍放黑框  top是要加多少像素先計算好
    new_im = cv2.cvtColor(new_im, cv2.COLOR_BGR2RGB)      #色彩空間，包括 RGB、HSI、HSL、HSV、HSB、YCrCb、CIE XYZ、CIE Lab8種
    return new_im

In [ ]:
train_frame = pd.read_csv(datapath+'/train.csv')
val_frame = pd.read_csv(datapath+'/val.csv')
test_frame = pd.read_csv(datapath+'/test.csv')

In [ ]:
train_frame

,Unnamed: 0,image_id,image_name,bb_x,bb_y,bb_width,bb_height,class_id,class_name,image_file,class_id_sp,class_name_sp,image_name_sp
0,0,0000139e-21dc-4d0c-bfe1-4cae3c85c829,0817/0000139e21dc4d0cbfe14cae3c85c829.jpg,83,59,128,228,817,Oak_Titmouse,0000139e-21dc-4d0c-bfe1-4cae3c85c829.png,817,Oak_Titmouse,0817/0000139e21dc4d0cbfe14cae3c85c829.jpg
1,1,01a472d8-e930-47a0-80aa-e4f958a2ef47,0817/01a472d8e93047a080aae4f958a2ef47.jpg,32,191,822,345,817,Oak_Titmouse,01a472d8-e930-47a0-80aa-e4f958a2ef47.png,817,Oak_Titmouse,0817/01a472d8e93047a080aae4f958a2ef47.jpg
2,2,036fba7c-9637-4635-8535-11ead2c1c728,0817/036fba7c96374635853511ead2c1c728.jpg,34,152,709,489,817,Oak_Titmouse,036fba7c-9637-4635-8535-11ead2c1c728.png,817,Oak_Titmouse,0817/036fba7c96374635853511ead2c1c728.jpg
3,3,07814887-f59b-44cb-9b7f-399999634fba,0817/07814887f59b44cb9b7f399999634fba.jpg,94,39,467,395,817,Oak_Titmouse,07814887-f59b-44cb-9b7f-399999634fba.png,817,Oak_Titmouse,0817/07814887f59b44cb9b7f399999634fba.jpg
4,4,08228657-41de-4312-8a6a-6c8897387975,0817/0822865741de43128a6a6c8897387975.jpg,230,57,754,550,817,Oak_Titmouse,08228657-41de-4312-8a6a-6c8897387975.png,817,Oak_Titmouse,0817/0822865741de43128a6a6c8897387975.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36414,48326,ae0e3680-ff8d-4937-974a-2cdc6631ddd7,0944/ae0e3680ff8d4937974a2cdc6631ddd7.jpg,170,141,125,244,944,Blue-headed_Vireo,ae0e3680-ff8d-4937-974a-2cdc6631ddd7.png,944,Blue-headed_Vireo,0944/ae0e3680ff8d4937974a2cdc6631ddd7.jpg
36415,48327,b618d50f-2a60-4158-a2b6-320b18f88437,0944/b618d50f2a604158a2b6320b18f88437.jpg,484,234,254,180,944,Blue-headed_Vireo,b618d50f-2a60-4158-a2b6-320b18f88437.png,944,Blue-headed_Vireo,0944/b618d50f2a604158a2b6320b18f88437.jpg
36416,48328,b631a841-ef53-48d3-b93e-a17fa5096cb7,0944/b631a841ef5348d3b93ea17fa5096cb7.jpg,101,123,258,456,944,Blue-headed_Vireo,b631a841-ef53-48d3-b93e-a17fa5096cb7.png,944,Blue-headed_Vireo,0944/b631a841ef5348d3b93ea17fa5096cb7.jpg
36417,48329,b6f3adc4-1de6-4eda-9e46-e1ff56a64454,0944/b6f3adc41de64eda9e46e1ff56a64454.jpg,87,121,236,422,944,Blue-headed_Vireo,b6f3adc4-1de6-4eda-9e46-e1ff56a64454.png,944,Blue-headed_Vireo,0944/b6f3adc41de64eda9e46e1ff56a64454.jpg


In [ ]:
for idx in np.arange(len(train_frame)):
    z = train_frame.iloc[idx]
    image = cv2.imread('/content/gdrive/My Drive/Bird_ID_project/nabirds/images/' + z['image_name'])
    cropped = image[z['bb_y']:(z['bb_y']+z['bb_height']), z['bb_x']:(z['bb_x']+z['bb_width'])]
    cv2.imwrite('/content/gdrive/My Drive/Bird_ID_project/nabirds/images2/'+z['image_name'],paint_to_square(cropped, z))
    if idx % 100 == 0:
        print("Processing image {}.".format(idx))  

Processing image 0.
Processing image 100.
Processing image 200.
Processing image 300.
Processing image 400.
Processing image 500.
Processing image 600.
Processing image 700.
Processing image 800.
Processing image 900.
Processing image 1000.
Processing image 1100.
Processing image 1200.
Processing image 1300.
Processing image 1400.
Processing image 1500.
Processing image 1600.
Processing image 1700.
Processing image 1800.
Processing image 1900.
Processing image 2000.
Processing image 2100.
Processing image 2200.
Processing image 2300.
Processing image 2400.
Processing image 2500.
Processing image 2600.
Processing image 2700.
Processing image 2800.
Processing image 2900.
Processing image 3000.
Processing image 3100.
Processing image 3200.
Processing image 3300.
Processing image 3400.
Processing image 3500.
Processing image 3600.
Processing image 3700.
Processing image 3800.
Processing image 3900.
Processing image 4000.
Processing image 4100.
Processing image 4200.
Processing image 4300.


In [ ]:
for idx in np.arange(len(val_frame)):
    z = val_frame.iloc[idx]
    image = cv2.imread('/content/gdrive/My Drive/Bird_ID_project/nabirds/images/' + z['image_name'])
    cropped = image[z['bb_y']:(z['bb_y']+z['bb_height']), z['bb_x']:(z['bb_x']+z['bb_width'])]
    cv2.imwrite('/content/gdrive/My Drive/Bird_ID_project/nabirds/images2/'+z['image_name'],paint_to_square(cropped, z))
    if idx % 100 == 0:
        print("Processing image {}.".format(idx))  

In [ ]:
for idx in np.arange(len(test_frame)):
    z = test_frame.iloc[idx]
    image = cv2.imread('/content/gdrive/My Drive/Bird_ID_project/nabirds/images/' + z['image_name'])
    cropped = image[z['bb_y']:(z['bb_y']+z['bb_height']), z['bb_x']:(z['bb_x']+z['bb_width'])]
    cv2.imwrite('/content/gdrive/My Drive/Bird_ID_project/nabirds/images2/'+z['image_name'],paint_to_square(cropped, z))
    if idx % 100 == 0:
        print("Processing image {}.".format(idx))  

In [ ]:
#full_frame.loc[full_frame['class_id']==814,'class_name'].iloc[0]
#Bird_list=full_frame[full_frame['class_id'].isin((full_frame.class_id.value_counts()==120).index[:25].tolist())].class_name.unique()
#print(Bird_list)
#print(len(Bird_list))
#y, X = make_npy_many(full_frame, Bird_list,datapath + '/data/y_30.npy', datapath + '/data/X_30.npy', datapath + '/data/L_30.npy')
#make_npy(df, 'Black_Throated_Sparrow', 'Harris_Sparrow', )

In [ ]:
# train_frame
#Bird_list=train_frame.class_name_sp.unique()
#Bird_id=train_frame.class_id_sp.unique()
# print(Bird_list, Bird_id)
#Birds = dict(zip(Bird_id, Bird_list))

In [ ]:
# def make_npy_many(df, bird_list, y_path, X_path, L_path):
#     print(y_path)
#     df = df[df['class_name'].isin(bird_list)]
#     df['class_id_2'] = pd.Series(np.zeros(len(df)), index=df.index)
#     var = df.class_id.unique()
#     print(var)
#     for i, z in enumerate(var):
#         df.loc[df['class_id']==z,'class_id_2'] = i
#         print('Now processing bird class = {}: {} '.format(i,df.loc[df['class_id']==z,'class_name'].iloc[0]))
#     target = df['class_id_2'].values.astype(int)
#     np.save(y_path, target)                                 #Y是存類別


#     zipper = zip(df.image_name, df.bb_y, df.bb_height, df.bb_x, df.bb_width)
#     new_x = []
#     for i, z in enumerate(zipper):
#         print(datapath + '/images/' + z[0])
#         image = cv2.imread(datapath + '/images/' + z[0])
#         cropped = image[z[1]:(z[1]+z[2]), z[3]:(z[3]+z[4])]   #crop_img = img[y:y+h, x:x+w]
#         new_x.append(paint_to_square(cropped, z)/255.0)       #將圖片附近增加黑色
#         print(i)
#         if (i < 10):
#             print(image.shape)
#             print(cropped.shape)
#             print(z[1])
#             print(z[2])
#             print(z[3])
#             print(z[4])
#             cropped = paint_to_square(cropped, z)
#             fig,ax = plt.subplots(1)
#             plt.imshow(cropped)
#             plt.show()
#         if (i == 10):
#           break

#     X = np.array(new_x)                                        
#     np.save(X_path, X)                                        #X是存裁切的圖片
    
#     np.save(L_path, bird_list)                                #L是存鳥的明細
    
#     return target, X

In [ ]:
# # create (cats, dogs) for (train, validation, test) dir
# categories = ['{}'.format(i).zfill(4) for i in df['id']]      #['0753', '0772', '0852', '0822']
# base_dir = '/content/gdrive/My Drive/Bird_ID_project/nabirds/images2/'
# createDir(base_dir)

# dirnames = {}
# for j, d in enumerate(categories):
#     dirname = d
#     dirpath = os.path.join(base_dir, d)
#     dirnames[dirname] = dirpath         #test_0753': '/content/gdrive/My Drive/Bird_ID_project/nabirds/data/test/0753

# print("# Creating directories")               #產生目錄
# for k, v in dirnames.items():
#     print("{} : {}".format(k,v))
#     createDir(v)